In [25]:
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
import pandas as pd


def read_data_from_bigquery(project_id, dataset_id, table_id, deviceId, start_date, end_date, limit=10):
    # Construct a BigQuery client object
    client = bigquery.Client()

    # Define the query to read data with filters for imei and date range
    query = f"""
        SELECT deviceId, hrlfc, totalDistance
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE deviceId = '{deviceId}' 
        AND DATE(eDate) BETWEEN '{start_date}' AND '{end_date}';
    """

    try:
        # Run the query and get the result
        query_job = client.query(query)
        results = query_job.result()

        # Collect and return the rows as a list of dictionaries
        rows = [dict(row) for row in results]
        return rows

    except GoogleAPIError as e:
        print("An error occurred:", e)
        return []


if __name__ == "__main__":
    # Replace with your project_id, dataset_id, and table_id
    project_id = "conn-datalake-prod"
    dataset_id = "bronze_telematics"
    table_id = "canbs4_pkt"

    # Specify IMEI and date range
    deviceId = "861557068886131"  # Example IMEI
    start_date = "2024-10-01"  # Example start date
    end_date = "2024-10-31"  # Example end date

    # Fetch data from BigQuery
    data = read_data_from_bigquery(project_id, dataset_id, table_id, deviceId, start_date, end_date)

    # Convert the data to a DataFrame
    df = pd.DataFrame(data)

    # Print the DataFrame
    print(df)


DefaultCredentialsError: File C:\Users\Ribish\AppData\Roaming\gcloud\application_default_credentials.json was not found.

In [24]:
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
import pandas as pd

scopes = ["https://www.googleapis.com/auth/cloud-platform"]
client = bigquery.Client(project='conn-datalake-prod')

def read_latest_packet_from_bigquery(project_id, dataset_id, table_id, imei, start_date, end_date):
    """Fetch the latest (most recent) packet for the given IMEI."""
    client = bigquery.Client()

    query = f"""
        SELECT *
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE deviceId = '{imei}'
        AND DATE(eDate) BETWEEN '{start_date}' AND '{end_date}'
    """

    try:
        query_job = client.query(query)
        results = query_job.result()
        rows = [dict(row) for row in results]
        return rows

    except GoogleAPIError as e:
        print(f"An error occurred while fetching data for IMEI {imei}: {e}")
        return []

def save_to_excel(data_dict, file_name):
    """Save data to an Excel file with each IMEI on a separate sheet."""
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        for imei, data in data_dict.items():
            df = pd.DataFrame(data)
            print(f"Saving latest packet for IMEI {imei}: {len(df)} row(s), {len(df.columns)} columns")
            df.to_excel(writer, sheet_name=f"IMEI_{imei}", index=False)
    print(f"Data saved to {file_name}")

if __name__ == "__main__":
    project_id = "conn-datalake-prod"
    dataset_id = "bronze_telematics"
    table_id = "canbs4_pkt"

    imei_list = ['359207068291762']
    start_date = '2025-04-03'
    end_date = '2025-04-03'

    data_dict = {}

    for imei in imei_list:
        print(f"Fetching latest packet for IMEI {imei}...")
        data = read_latest_packet_from_bigquery(project_id, dataset_id, table_id, imei, start_date, end_date)
        data_dict[imei] = data

    excel_file_name = "abishek.xlsx"
    save_to_excel(data_dict, excel_file_name)


DefaultCredentialsError: File C:\Users\Ribish\AppData\Roaming\gcloud\application_default_credentials.json was not found.

In [2]:
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
import pandas as pd
scopes = ["https://www.googleapis.com/auth/cloud-platform"]
client = bigquery.Client(project='conn-datalake-prod')

def read_data_from_bigquery(project_id, dataset_id, table_id, imei_list, start_date, end_date):
    # Construct a BigQuery client object
    client = bigquery.Client()

    # Create a query string to fetch data for the given IMEIs within the specified date range
    imei_str = ', '.join(f"'{imei}'" for imei in imei_list)  # Formatting IMEIs for SQL query
    query = f"""
        SELECT deviceId,eDateTime,sequenceNumber,longitude,longitude,utc,hrlfc,topGear,sweetSpotPercent,seconds,minute,hour,month,day,year,totalDistance,fuelLevel, engineSpeed, engineOperatingHours, vehicleSpeed, engineOilPressure,	engineCoolantTemp,accPedalPosition,live,ignitionStatus,sFuelLevelLtrs,sFuelLevelPer
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE deviceId IN ({imei_str})
        AND DATE(eDate) BETWEEN '{start_date}' AND '{end_date}';
    """

    try:
        # Run the query and get the result
        query_job = client.query(query)
        results = query_job.result()

        # Collect and return the rows as a list of dictionaries
        rows = [dict(row) for row in results]
        return rows

    except GoogleAPIError as e:
        print("An error occurred:", e)
        return []

def save_to_excel(data, file_name):
    # Convert data to a DataFrame
    df = pd.DataFrame(data)
    print(f"Number of columns: {len(df.columns)}")
    # Save the DataFrame to an Excel file
    df.to_excel(file_name, index=False)
    print(f"Column names: {', '.join(df.columns)}")
    print(f"Data saved to {file_name}")

if __name__ == "__main__":
    # Replace with your project_id, dataset_id, and table_id
    project_id = "conn-datalake-prod"
    dataset_id = "bronze_telematics"
    table_id = "canbs4_pkt"  # Update with your actual table ID

    # List of IMEIs to fetch data for
    imei_list = ['352467113127596','352467113210111']
    
    # Define the date range
    start_date = '2024-10-01'
    end_date = '2025-03-31'

    # Fetch data from BigQuery for specified IMEIs within the date range
    data = read_data_from_bigquery(project_id, dataset_id, table_id, imei_list, start_date, end_date)
    
    # Check if data was fetched successfully
    if data:
        # Define the output Excel file name
        excel_file_name = "LocationData.xlsx"

        # Save the data to an Excel file
        save_to_excel(data, excel_file_name)

    else:
        print("No data found.")


Number of columns: 27
Column names: deviceId, eDateTime, sequenceNumber, longitude, longitude_1, utc, hrlfc, topGear, sweetSpotPercent, seconds, minute, hour, month, day, year, totalDistance, fuelLevel, engineSpeed, engineOperatingHours, vehicleSpeed, engineOilPressure, engineCoolantTemp, accPedalPosition, live, ignitionStatus, sFuelLevelLtrs, sFuelLevelPer
Data saved to LocationData.xlsx


In [ ]:
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
import pandas as pd

scopes = ["https://www.googleapis.com/auth/cloud-platform"]
client = bigquery.Client(project='conn-datalake-prod')

def read_data_from_bigquery(project_id, dataset_id, table_id, imei, start_date, end_date):
    # Construct a BigQuery client object
    client = bigquery.Client()

    # Create a query string to fetch data for the given IMEI within the specified date range
    query = f"""
        SELECT deviceId,eDateTime,sequenceNumber,latitude,longitude,utc,hrlfc,topGear,sweetSpotPercent,seconds,minute,hour,month,day,year,totalDistance,fuelLevel, engineSpeed, engineOperatingHours, vehicleSpeed, engineOilPressure,	engineCoolantTemp,accPedalPosition,live,ignitionStatus,sFuelLevelLtrs,sFuelLevelPer
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE deviceId = '{imei}'
        AND DATE(eDate) BETWEEN '{start_date}' AND '{end_date}'
        ;
    """

    try:
        # Run the query and get the result
        query_job = client.query(query)
        results = query_job.result()

        # Collect and return the rows as a list of dictionaries
        rows = [dict(row) for row in results]
        return rows

    except GoogleAPIError as e:
        print(f"An error occurred while fetching data for IMEI {imei}: {e}")
        return []

def save_to_excel(data_dict, file_name):
    # Save the data to an Excel file with multiple sheets
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        for imei, data in data_dict.items():
            # Convert data to a DataFrame
            df = pd.DataFrame(data)
            print(f"Saving data for IMEI {imei}: {len(df)} rows, {len(df.columns)} columns")
            # Save each IMEI's data to a separate sheet
            df.to_excel(writer, sheet_name=f"IMEI_{imei}", index=False)
    print(f"Data saved to {file_name}")

if __name__ == "__main__":
    # Replace with your project_id, dataset_id, and table_id
    project_id = "conn-datalake-prod"
    dataset_id = "bronze_telematics"
    table_id = "canbs4_pkt"  # Update with your actual table ID

    # List of IMEIs to fetch data for
    imei_list = ['352467113273630']
    # Define the date range
    start_date = '2025-03-01'
    end_date = '2025-03-31'

    # Dictionary to store data for each IMEI
    data_dict = {}

    # Fetch data for each IMEI and store it in the dictionary
    for imei in imei_list:
        print(f"Fetching data for IMEI {imei}...")
        data = read_data_from_bigquery(project_id, dataset_id, table_id, imei, start_date, end_date)
        data_dict[imei] = data

    # Define the output Excel file name
    excel_file_name = "352467113273630.xlsx"

    # Save all data to an Excel file with multiple sheets
    save_to_excel(data_dict, excel_file_name)


Fetching data for IMEI 359207067650828...
Saving data for IMEI 359207067650828: 434 rows, 27 columns
Data saved to 359207067650828.xlsx


In [26]:
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
import pandas as pd

scopes = ["https://www.googleapis.com/auth/cloud-platform"]
client = bigquery.Client(project='conn-datalake-prod')

def read_data_from_bigquery(project_id, dataset_id, table_id, imei, start_date, end_date):
    # Construct a BigQuery client object
    client = bigquery.Client()

    # Create a query string to fetch data for the given IMEI within the specified date range
    query = f"""
        SELECT deviceId, eDateTime, engineOperatingHours, totalDistance
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE deviceId = '{imei}'
        AND DATE(eDate) BETWEEN '{start_date}' AND '{end_date}'
        ORDER BY eDate DESC
        LIMIT 1
        ;
    """

    try:
        # Run the query and get the result
        query_job = client.query(query)
        results = query_job.result()

        # Collect and return the rows as a list of dictionaries
        rows = [dict(row) for row in results]
        return rows

    except GoogleAPIError as e:
        print(f"An error occurred while fetching data for IMEI {imei}: {e}")
        return []

def save_to_excel(data_dict, file_name):
    # Save the data to an Excel file with multiple sheets
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        for imei, data in data_dict.items():
            # Convert data to a DataFrame
            df = pd.DataFrame(data)
            print(f"Saving data for IMEI {imei}: {len(df)} rows, {len(df.columns)} columns")
            # Save each IMEI's data to a separate sheet
            df.to_excel(writer, sheet_name=f"IMEI_{imei}", index=False)
    print(f"Data saved to {file_name}")

if __name__ == "__main__":
    # Replace with your project_id, dataset_id, and table_id
    project_id = "conn-datalake-prod"
    dataset_id = "bronze_telematics"
    table_id = "canbs4_pkt"  # Update with your actual table ID

    # List of IMEIs to fetch data for
    imei_list = ['359207066348721','359207066347285','359207066297837','359207066344894',
'359207066493279','359207066429612','359207066347525','359207066078393',
'359207066265032','359207066265776','359207066358423','359207066270685',
'359207066804640','359207066104397','352467110967788','352467111019530',
'359207066286343','359207066324847','359207066355502','359207066114297',
'359207066087956','359207066269836','352467111373929','359207067097087',
'352467111348988','352467111410747','352467111407107','352467111475351',
'352467111444845','352467111086299','352467111423740','352467111399957',
'352467111448796','352467111357237','352467111390378','352467111471293',
'352467111469552','352467111426834','352467111427998','352467111425869',
'352467111373705','352467111438920','352467111390725','359207067110476',
'352467111474990','352467111380478','352467111408535','352467111478728',
'352467111085572','352467111343518','352467111448192','352467110994600',
'352467111400102','359207066940907','352467111406216','352467111426479',
'352467111400110','352467111469479','352467111399734','352467111389628',
'352467111403874','352467111407966','352467111362872','352467110978447',
'352467111408923','352467111376096','352467111388349','352467111372699',
'352467111385212','359207067100790','352467111406554','352467111016288',
'352467111422304','352467111419508','352467111393919','359207067282184',
'359218066030540','359207067823029','359207068174307','359207068124948',
'359207068270386','359207068069333','359207068208667','359207068208774',
'359207068217411','359207068126737','359207068175239','359207068448776',
'359207068508322','359207068385846','359207068448925','359207068475068',
'359207068476546','359207068277480','352467112689356','352467112705350',
'359207068428018','359207068458486','359207068340916','359207068353265',
'359207068347002','359207068368669','359207068361441','359207068368933',
'359207068318318','359207068477114','359207068348364','359207068353596',
'359207068395506','352467112684647','359207068219391','359207068302874',
'359207068416864','359207068414356','359207068510401','359207068461662',
'359207068318573','352467112683060','359207068341260','359207068449212',
'359207068351889','359207068324720','359207068437696','359207068488541',
'359207068431566','359207068522828','359207068524550','359207068457371',
'359207068520749','359207068616299','359207068623345','359207068528049',
'359207068633823','359207068629920','359207068521101','359207068486503',
'359207068617453','352467113106350','352467113135979','352467113123728',
'352467113095876','352467113095579','352467113081736','352467113198100',
'352467113220680','352467113200096','352467113413822','352467112904276',
'352467112953232','352467112828111','352467113020130','352467113409143',
'352467113037522','352467113026079','352467113020841','352467112953265',
'352467112953331','352467112903112','352467112825117','352467113413954',
'352467112953539','352467112953612','352467113020098','352467113343516',
'352467113014943','352467113406503','352467113038017','352467113100007',
'352467113092477','352467113108364','352467113322627','352467113106228',
'352467113107697','352467113123827','352467113106202','352467113406388',
'352467113106491','352467113106590','352467113413053','352467113108216',
'352467113100577','352467113136449','352467113099886','352467113105733',
'352467113099092','352467113108430','352467113107754','352467113224666',
'352467113110741','352467113099134','352467113100205','352467113004233',
'352467113094937','352467113099423','352467113127661','352467113087022',
'352467113004837','352467113091818','352467113100031','352467113101690',
'352467113094408','352467113099746','352467113106582','352467113102300',
'352467113127091','352467113409986','352467113322783','352467113399278',
'352467113313527','352467113399492','352467113319342','352467113322635',
'352467113323740','352467113322650','352467113322767','352467113322775',
'352467113319383','352467113319151','352467113399534','352467113319326',
'352467113313592','352467113313626','352467113324276','352467113323385',
'352467113399385','352467113106707','352467113248863','352467113117647',
'352467113197300','352467113117704','352467113192061','352467113229889',
'352467113195536','352467113295526','352467113200302','352467113328152',
'352467113324284','352467113335967','352467113402098','352467113410240',
'352467113331099','352467113318229','352467113413509','352467113327576',
'352467113222488','352467113200765','352467113204973','352467113148162',
'352467113199157','352467113201656','352467113231091','352467113221076',
'352467113219195','352467113214998','352467113229871','352467113225614',
'352467113225069','352467113195924','352467113212810','352467113215276',
'352467113191352','352467113215169','352467113229210','352467113408343',
'352467113195882','352467113202001','352467113229640','352467113191451',
'352467113229848','352467113215508','352467113089754','352467113312420',
'352467113327618','352467113345370','352467113314103','352467113345081',
'352467113330018','352467113413905','352467113317270','352467113313808',
'352467113413541','352467113346832','352467113343565','352467113227784',
'352467113761188','352467113325380','352467113317767','352467113317411',
'352467113313121','352467113320126','352467113361583','352467113319532',
'352467113320324','352467113361500','352467113348309','352467113313170',
'352467113323708','352467113402163','352467113313097','352467113413434',
'352467113317387','352467113319623','352467113329978','352467113352798',
'352467113317585','352467113317486','352467113813237','352467113748722',
'352467113687235','352467113791532','352467113747187','352467113729847',
'352467113690577','352467113798594','352467113808062','352467113815091',
'352467113808088','352467113722008','352467113761709','352467113780055',
'352467113815703','352467113494186','352467113712496','352467113800879',
'352467113729540','352467113768977','352467113753045','352467113834977']

    # Define the date range
    start_date = '2025-04-08'
    end_date = '2025-04-08'

    # Dictionary to store data for each IMEI
    data_dict = {}

    # Fetch data for each IMEI and store it in the dictionary
    for imei in imei_list:
        print(f"Fetching data for IMEI {imei}...")
        data = read_data_from_bigquery(project_id, dataset_id, table_id, imei, start_date, end_date)
        data_dict[imei] = data

    # Define the output Excel file name
    excel_file_name = "kccl.xlsx"

    # Save all data to an Excel file with multiple sheets
    save_to_excel(data_dict, excel_file_name)


DefaultCredentialsError: File C:\Users\Ribish\AppData\Roaming\gcloud\application_default_credentials.json was not found.

In [ ]:
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
import pandas as pd
import json
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from urllib.parse import quote

client = bigquery.Client(project='conn-datalake-prod')

CONFIG_FILEPATH = r'C:/Users/700023/Documents/mysql.json'
with open(CONFIG_FILEPATH) as config_file:
    mysql_config = json.load(config_file)

HOST = mysql_config["HOST"]
PORT = mysql_config["PORT"]
USER = mysql_config["USER"]
PASSWORD = mysql_config["PASSWORD"]
DATABASE = mysql_config["DATABASE"]

def connecting_to_mysql(ip_address, port, username, password, database_name):
    db_connection_str = f"mysql+pymysql://{username}:{quote(password)}@{ip_address}:{port}/{database_name}"
    engine = create_engine(db_connection_str)
    return engine  # Return engine for queries

# Connect to MySQL
engine = connecting_to_mysql(HOST, PORT, USER, PASSWORD, DATABASE)

def fetch_last_packet_from_table(project_id, dataset_id, table_id, imei, date, columns):
    """
    Fetches the last packet (latest eDateTime) for a given IMEI and date from a specific BigQuery table.
    Ensures partition filtering using eDate.
    """
    query = f"""
        SELECT {', '.join(columns)}
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE deviceId = '{imei}' AND eDate = '{date}'
        ORDER BY eDateTime DESC
        LIMIT 1
    """
    try:
        query_job = client.query(query)
        results = query_job.result()
        data = [dict(row) for row in results]
        return data[0] if data else None
    except GoogleAPIError as e:
        print(f"Error fetching data from {table_id} for IMEI {imei}: {e}")
    return None

def fetch_last_nonzero_dpf_packet(project_id, dataset_id, table_id, imei, date, columns):
    """
    Fetches the latest packet where particulateTrap1SootLoadPercent is NOT 0.
    If all values are 0, returns the most recent packet.
    """
    query = f"""
        WITH latest_packets AS (
            SELECT {', '.join(columns)}
            FROM `{project_id}.{dataset_id}.{table_id}`
            WHERE deviceId = '{imei}' AND eDate = '{date}'
            ORDER BY eDateTime DESC
        )
        SELECT * FROM latest_packets
        WHERE particulateTrap1SootLoadPercent != 0
        LIMIT 1
    """
    try:
        query_job = client.query(query)
        results = query_job.result()
        data = [dict(row) for row in results]
        if data:
            return data[0]
        fallback_query = f"""
            SELECT {', '.join(columns)}
            FROM `{project_id}.{dataset_id}.{table_id}`
            WHERE deviceId = '{imei}' AND eDate = '{date}'
            ORDER BY eDateTime DESC
            LIMIT 1
        """
        query_job = client.query(fallback_query)
        results = query_job.result()
        data = [dict(row) for row in results]
        return data[0] if data else None
    except GoogleAPIError as e:
        print(f"Error fetching data from {table_id} for IMEI {imei}: {e}")
    return None

def fetch_imei_to_chassis_mapping(engine, imei_list):
    """Fetches chassis number (vin) mapped to new_device (deviceId) from MySQL."""
    try:
        query = f"""
            SELECT vin, new_device 
            FROM vecvdb.vin_device_mapping 
            WHERE new_device IN ({', '.join(map(str, imei_list))})
        """
        df = pd.read_sql(query, engine)
        return dict(zip(df["new_device"], df["vin"]))
    except SQLAlchemyError as e:
        print(f"Error fetching chassis mapping: {e}")
        return {}
def fetch_chassis_and_reg_no_mapping(engine, imei_list):
    """Fetches chassis number (vin) and registration number (reg_no) mapped to new_device (deviceId) from MySQL."""
    try:
        query = f"""
            SELECT vdm.vin AS chassis_no, ucm.reg_no
            FROM vecvdb.vin_device_mapping vdm
            LEFT JOIN vecvdb.UEL_custid_master ucm ON vdm.vin = ucm.Chassis_no
            WHERE vdm.new_device IN ({', '.join(map(str, imei_list))})
        """
        df = pd.read_sql(query, engine)
        return {row["chassis_no"]: row["reg_no"] for _, row in df.iterrows()}
    except SQLAlchemyError as e:
        print(f"Error fetching chassis and reg_no mapping: {e}")
        return {}

def merge_data(imei, canbs4_data, can2bs6_data, chassis_mapping, reg_no_mapping):
    """
    Merges data from both tables on deviceId and adds registration number.
    """
    if not canbs4_data:
        print(f"No data found in canbs4_pkt for IMEI {imei}")
        return None
    if not can2bs6_data:
        print(f"No data found in can2bs6 for IMEI {imei}")

    chassis_no = chassis_mapping.get(imei, "Unknown")
    reg_no = reg_no_mapping.get(chassis_no, "Unknown")

    merged_data = {
        "deviceId": canbs4_data["deviceId"],
        "Chassis_no": chassis_no,
        "Reg_no": reg_no,  # Adding Registration Number
        "eDateTime": canbs4_data["eDateTime"],
        "FuelinLitres": canbs4_data["sfuelLevelltrs"],
        "DEF Level": can2bs6_data["sFuelLevelLtrs"] if can2bs6_data else None,
        "DPF": can2bs6_data["particulateTrap1SootLoadPercent"] if can2bs6_data else None
    }
    return merged_data


def save_to_excel(data_list, file_name):
    """
    Saves the final merged data to an Excel file and adds a note for DPF=0.
    """
    df = pd.DataFrame(data_list)
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Last Packet Data', index=False)
        workbook = writer.book
        sheet = writer.sheets['Last Packet Data']
        sheet.cell(row=len(df) + 3, column=1, value="Note: If DPF = 0, the vehicle is in stoppage mode.")
    print(f"Data saved to {file_name}")


if __name__ == "__main__":
    project_id = "conn-datalake-prod"
    dataset_id = "bronze_telematics"
    imei_list = ['359207067842748', '359207068159761']
    date = '2025-02-28'

    # Fetch mappings
    chassis_mapping = fetch_imei_to_chassis_mapping(engine, imei_list)
    reg_no_mapping = fetch_chassis_and_reg_no_mapping(engine, imei_list)

    final_data = []
    for imei in imei_list:
        print(f"Processing IMEI {imei}...")
        canbs4_data = fetch_last_packet_from_table(project_id, dataset_id, "canbs4_pkt", imei, date, ["deviceId", "eDateTime", "sfuelLevelltrs"])
        can2bs6_data = fetch_last_nonzero_dpf_packet(project_id, dataset_id, "can2bs6_pkt", imei, date, ["deviceId", "sFuelLevelLtrs", "particulateTrap1SootLoadPercent"])
        
        merged_data = merge_data(imei, canbs4_data, can2bs6_data, chassis_mapping, reg_no_mapping)
        
        if merged_data:
            final_data.append(merged_data)

    if final_data:
        save_to_excel(final_data, "l_packet_data.xlsx")
    else:
        print("No valid data to save.")

        print("No valid data to save.")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Ribish/Downloads/Karan/mysql.json'

In [5]:
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
import pandas as pd
import json
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from urllib.parse import quote

# BigQuery Client
client = bigquery.Client(project='conn-datalake-prod')

# Load MySQL Config
CONFIG_FILEPATH = 'C:/Users/Ribish/Downloads/Karan/mysql.json'
with open(CONFIG_FILEPATH) as config_file:
    mysql_config = json.load(config_file)

HOST = mysql_config["HOST"]
PORT = mysql_config["PORT"]
USER = mysql_config["USER"]
PASSWORD = mysql_config["PASSWORD"]
DATABASE = mysql_config["DATABASE"]

def connecting_to_mysql(ip_address, port, username, password, database_name):
    db_connection_str = f"mysql+pymysql://{username}:{quote(password)}@{ip_address}:{port}/{database_name}"
    engine = create_engine(db_connection_str)
    return engine  # Return engine for queries

# Connect to MySQL
engine = connecting_to_mysql(HOST, PORT, USER, PASSWORD, DATABASE)

def fetch_last_packet_from_table(project_id, dataset_id, table_id, imei, date, columns):
    """Fetches the last packet (latest eDateTime) for a given IMEI and date from a BigQuery table."""
    query = f"""
        SELECT {', '.join(columns)}
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE deviceId = '{imei}' AND eDate = '{date}'  -- Ensuring partition filtering
        ORDER BY eDateTime DESC
        LIMIT 1
    """
    try:
        query_job = client.query(query)
        results = query_job.result()
        data = [dict(row) for row in results]
        return data[0] if data else None  # Return the latest packet or None if no data
    except GoogleAPIError as e:
        print(f"Error fetching data from {table_id} for IMEI {imei}: {e}")
    return None  # Return None if an error occurs

def fetch_imei_to_chassis_mapping(engine, imei_list):
    """Fetches chassis number (vin) mapped to new_device (deviceId) from MySQL."""
    try:
        query = f"""
            SELECT vin, new_device 
            FROM vecvdb.vin_device_mapping 
            WHERE new_device IN ({', '.join(map(str, imei_list))})
        """
        df = pd.read_sql(query, engine)
        return dict(zip(df["new_device"], df["vin"]))  # Mapping deviceId to chassisNo
    except SQLAlchemyError as e:
        print(f"Error fetching chassis mapping: {e}")
        return {}

def merge_data(imei, canbs4_data, can2bs6_data, chassis_mapping):
    """Merges data from both tables on deviceId and renames columns as per requirement."""
    if not canbs4_data:
        print(f"No data found in canbs4_pkt for IMEI {imei}")
        return None
    if not can2bs6_data:
        print(f"No data found in can2bs6 for IMEI {imei}")

    merged_data = {
        "deviceId": canbs4_data["deviceId"],
        "chassisno": chassis_mapping.get(imei, "Unknown"),  # Map chassis number
        "eDateTime": canbs4_data["eDateTime"],
        "FuelinLitres": canbs4_data["sfuelLevelltrs"],
        "DEF Level": can2bs6_data["sFuelLevelLtrs"] if can2bs6_data else None,
        "DPF": can2bs6_data["particulateTrap1SootLoadPercent"] if can2bs6_data else None
    }
    return merged_data

def save_to_excel(data_list, file_name):
    """Saves the final merged data to an Excel file and adds a note for DPF=0."""
    df = pd.DataFrame(data_list)
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Last Packet Data', index=False)

        # Adding a note if DPF is 0
        workbook = writer.book
        sheet = writer.sheets['Last Packet Data']
        sheet.cell(row=len(df) + 3, column=1, value="Note: If DPF = 0, the vehicle is in stoppage mode.")

    print(f"Data saved to {file_name}")

if __name__ == "__main__":
    project_id = "conn-datalake-prod"
    dataset_id = "bronze_telematics"
    imei_list = ['359207067857811', '359207067846152', '359207067861714', '359207067864619']
    date = '2025-02-27'  # Ensure this matches eDate partition

    # Fetch chassis number mapping
    chassis_mapping = fetch_imei_to_chassis_mapping(engine, imei_list)

    final_data = []

    for imei in imei_list:
        print(f"Processing IMEI {imei}...")

        # Fetch the last packet from both tables
        canbs4_columns = ["deviceId", "eDateTime", "sfuelLevelltrs"]
        can2bs6_columns = ["deviceId", "sFuelLevelLtrs", "particulateTrap1SootLoadPercent"]

        canbs4_data = fetch_last_packet_from_table(project_id, dataset_id, "canbs4_pkt", imei, date, canbs4_columns)
        can2bs6_data = fetch_last_packet_from_table(project_id, dataset_id, "can2bs6_pkt", imei, date, can2bs6_columns)

        # Merge and store
        merged_data = merge_data(imei, canbs4_data, can2bs6_data, chassis_mapping)
        if merged_data:
            final_data.append(merged_data)

    # Save to Excel
    if final_data:
        save_to_excel(final_data, "last1_packet_data.xlsx")
    else:
        print("No valid data to save.")


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Ribish/Downloads/Karan/mysql.json'

In [14]:
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
import pandas as pd

scopes = ["https://www.googleapis.com/auth/cloud-platform"]
client = bigquery.Client(project='conn-datalake-prod')

def read_data_from_bigquery(project_id, dataset_id, table_id, imei_list, start_date, end_date):
    # Construct a BigQuery client object
    client = bigquery.Client()

    results_summary = []
    
    for imei in imei_list:
        query = f"""
            SELECT COUNT(*) AS record_count
            FROM `{project_id}.{dataset_id}.{table_id}`
            WHERE deviceId = '{imei}'
            AND DATE(eDate) BETWEEN '{start_date}' AND '{end_date}';
        """

        try:
            # Run the query and get the result
            query_job = client.query(query)
            results = query_job.result()
            for row in results:
                record_count = row["record_count"]
                status = "Data Found" if record_count > 0 else "Data Not Found"
                results_summary.append({"deviceId": imei, "Status": status})
        
        except GoogleAPIError as e:
            print(f"An error occurred while processing IMEI {imei}: {e}")
            results_summary.append({"deviceId": imei, "Status": "Error"})
    
    return results_summary

def save_to_excel(data, file_name):
    # Convert data to a DataFrame
    df = pd.DataFrame(data)

    # Save the DataFrame to an Excel file
    df.to_excel(file_name, index=False)
    print(f"Data saved to {file_name}")

if __name__ == "__main__":
    # Replace with your project_id, dataset_id, and table_id
    project_id = "conn-datalake-prod"
    dataset_id = "bronze_telematics"
    table_id = "canbs4_pkt"  # Update with your actual table ID

    # List of IMEIs to fetch data for (replace with your list of 600 chassis numbers)
    imei_list = ['351510091728017','353701094346059','353701097557231','353701098204965',
'353701098217645','353701098219955','357789641760282','357789642806803',
'358735076435220','358735077692084','358735078412854','358735078916235',
'358735078922993','358735079079272','358735079083308','358735079178041',
'358735079180526','358735079200506','358735079208939','358735079211941',
'358735079213384','358735079214382','358735079672068','358735079687983',
'358735079757315','358735079778659','358735079778832','358735079785589',
'358735079785910','358735079795406','862292053229548','862292053230074',
'862292053239281','862292053241022','862292053241345','862292053892626',
'862292053892717','862292053892725','862292053892949','862292053893004',
'862292053893095','862292053895090','862292053895132','862292053895157',
'862292053895579','862292053895694','862292053895710','862292053897369',
'862292053898250','862292053898664','862292053898698','865784051017187',
'865784051030321','865784051042235','865784051330291','865784051340233',
'865784051341033','865784051343625','865784051353186','865784054647675',
'865784054647865','865784054648145','865784054648236','865784054648293',
'865784054648632','865784054648640','865784054648798','865784054649267',
'865784054649275','865784054649390','865784054649523','865784054650513',
'865784054650786','865784054650927','865784054651081','865784054651412',
'865784054651453','865784054651487','865784054651503','865784054651628',
'865784054651701','865784054651768','865784054651875','865784054651974',
'865784054652014','865784054652071','865784054652147','865784054652303',
'865784054896785','865784054896868','865784054896892','865784054896991',
'865784054897130','865784054897494','865784054897544','865784054897593',
'865784054897676','865784054897734','865784054897890','865784054898062',
'865784054898179','865784054898195','865784054898286','865784054898302',
'865784054898492','865784054898559','865784054898716','865784054898732',
'865784054898906','865784054898914','865784054899078','865784054899201',
'865784054899284','865784054899292','865784054899342','865784054899441',
'865784054899557','865784054899763','865784054899912','865784054900413',
'865784054900629','865784054900835','865784054901270','865784054901296',
'865784054901346','865784054901692','865784054902195','865784054902203',
'865784054902476','865784054902864','865784054903268','865784054903441',
'865784054903573','865784054903599','865784054903672','865784054903771',
'865784054904217','865784054904233','865784054904464','865784054904522',
'865784054904688','865784054904902','865784054905016','865784054905032',
'865784054905073','865784054905685','865784054905834','865784054905867',
'865784054906030','865784054906147','865784054906345','865784054906378',
'865784054906436','865784055316437','865784055316478','865784055316668',
'865784055316890','865784055317161','865784055317468','865784055317518',
'865784055317542','865784055317617','865784055317633','865784055317641',
'865784055317658','865784055317666','865784055317690','865784055317732',
'865784055317740','865784055317757','865784055317781','865784055317799',
'865784055317831','865784055317856','865784055317880','865784055317914',
'865784055317930','865784055317948','865784055317955','865784055318003',
'865784055318060','865784055318078','865784055318169','865784055318318',
'865784055318474','865784055318656','865784055318953','865784055318961',
'865784055318987','865784055318995','865784055319001','865784055319019',
'865784055319035','865784055319050','865784055319100','865784055319167',
'865784055319175','865784055319209','865784055319282','865784055319324',
'865784055319357','865784055319415','865784055319605','865784055319639',
'865784055319704','865784055319712','865784055319720','865784055319928',
'865784055319936','865784055319944','865784055319969','865784055320090',
'865784055320207','865784055320223','865784055320231','865784055320264',
'865784055320272','865784055320280','865784055320348','865784055320397',
'865784055320629','865784055320694','865784055320785','865784055320819',
'865784055320835','865784055320959','865784055320975','865784055320983',
'865784055321015','865784055321056','865784055321080','865784055321155',
'865784055321221','865784055321239','865784055321247','865784055321270',
'865784055321288','865784055321692','865784055321718','865784055321759',
'865784055321775','865784055321866','865784055321874','865784055321999',
'865784055322021','865784055322088','865784055322096','865784055322112',
'865784055322161','865784055322179','865784055322195','865784055322229',
'865784055322252','865784055322294','865784055322310','865784055322401',
'865784055322518','865784055322526','865784055322757','865784055322872',
'865784055322898','865784055322955','865784055323219','865784055323227',
'865784055323318','865784055323367','865784055323391','865784055323409',
'865784055323458','865784055323466','865784055323557','865784055323599',
'865784055323615','865784055323680','865784055323714','865784055323904',
'865784055324019','865784055324076','865784055324308','865784055324324',
'865784055324415','865784055324472','865784055324746','865784055324928',
'865784055324969','865784055324977','865784055325040','865784055325149',
'865784055325263','865784055325305','865784055325313','865784055325339',
'865784055325388','865784055325404','865784055325412','865784055325669',
'865784055325685','865784055325727','865784055325966','865784055326543',
'865784055779501','865784055781267','865784055781507','865784055783297',
'865784055786878','865784055797859','865784055797867','865784055797883',
'865784055797909','865784055797966','865784055797974','865784055797982',
'865784055798022','865784055798063','865784055798071','865784055798089',
'865784055798121','865784055798196','865784055798394','865784055798402',
'865784055798428','865784055798477','865784055798576','865784055798667',
'865784055798766','865784055798782','865784055798808','865784055798816',
'865784055799269','865784055799319','865784055799327','865784055799335',
'865784055799376','865784055799392','865784055799400','865784055799418',
'865784055799426','865784055799434','865784055799442','865784055799459',
'865784055799475','865784055799483','865784055799491','865784055799517',
'865784055799525','865784055799533','865784055799541','865784055799558',
'865784055799574','865784055799582','865784055799590','865784055799608',
'865784055799616','865784055799624','865784055799632','865784055799640',
'865784055799665','865784055799905','865784055800075','865784055800083',
'865784055800109','865784055800133','865784055800141','865784055800158',
'865784055800224','865784055800240','865784055800349','865784055800364',
'865784055800612','865784055800620','865784055800661','865784055800687',
'865784055800778','865784055800802','865784055800851','865784055800893',
'865784055800901','865784055800927','865784055800943','865784055800950',
'865784055801016','865784055801040','865784055801099','865784055801115',
'865784055801123','865784055801149','865784055801172','865784055801420',
'865784055801461','865784055801636','865784055801701','865784055801727',
'865784055801743','865784055801750','865784055801768','865784055801818',
'865784055801834','865784055801883','865784055801891','865784055801909',
'865784055802089','865784055802121','865784055802147','865784055802287',
'865784055802436','865784055802527','865784055803087','865784055803293',
'865784055803848','865784055803863','865784055804077','865784055804192',
'865784055804481','865784055804515','865784055804523','865784055804531',
'865784055804549','865784055804564','865784055804796','865784055804820',
'865784055804838','865784055804846','865784055804853','865784055804887',
'865784055805033','865784055805132','865784055805140','865784055805173',
'865784055805199','865784055805306','865784055805330','865784055805389',
'865784055805439','865784055805447','865784055805694','865784055805801',
'865784055805819','865784055805850','865784055805918','865784055806031',
'865784055806346','865784055806411','865784055806627','865784055806908',
'865784055807914','865784055807948','865784055808482','865784055808755',
'865784055808805','865784055808847','865784055808870','865784055808912',
'865784055811221','865784055811692','865784055811965','865784055812005',
'865784055812179','865784055812229','865784055812245','865784055812294',
'865784055813318','865784055815263','865784055815313','865784055815685',
'865784055815693','865784055815701','865784055815800','865784055816105',
'865784055817632','865784055818481','865784055818499','865784055818507',
'865784055818515','865784055818531','865784055818549','865784055818556',
'865784055818564','865784055818572','865784055818580','865784055818598',
'865784055818606','865784055818622','865784055818630','865784055818655',
'865784055818689','865784055818697','865784055818705','865784055818713',
'865784055818721','865784055818739','865784055818754','865784055818762',
'865784055818770','865784055818788','865784055818796','865784055818804',
'865784055818820','865784055818838','865784055818846','865784055818937',
'865784055819018','865784055819034','865784055819075','865784055819125',
'865784055819141','865784055819182','865784055819281','865784055819448',
'865784055819455','865784055819562','865784055819570','865784055819588',
'865784055819646','865784055819661','865784055819810','865784055819844',
'865784055819851','865784055819893','865784055819927','865784055819935',
'865784055819950','865784055819992','865784055820008','865784055820024',
'865784055820032','865784055820040','865784055820065','865784055820081',
'865784055820099','865784055820115','865784055820123','865784055820149',
'865784055820156','865784055820172','865784055820214','865784055820222',
'865784055820248','865784055820255','865784055820263','865784055820370',
'865784055820396','865784055820412','865784055820511','865784055820586',
'865784055820602','865784055820883','865784055820891','865784055820982',
'865784055821105','865784055821212','865784055821279','865784055821345',
'865784055821360','865784055821386','865784055821394','865784055821493',
'865784055821899','865784055822046','865784055822061','865784055822400',
'865784055822889','865784055822988','865784055823028','865784055823291',
'865784055823887','865784055823952','865784055824182','865784055824190',
'865784055824406','865784055824463','865784055824562','865784055824638',
'865784055824695','865784055824729','865784055824752','865784055824786',
'865784055824810','865784055824828','865784055824851','865784055824893',
'865784055824919','865784055825007','865784055825221','865784055825239',
'865784055825296','865784055825650','865784055825825','865784055826435',
'865784055826625','865784055826708','865784055826765','865784055827243',
'865784055827250','865784055827466','865784055827482','865784055827540',
'865784055827912','865784055828043','865784055828209','865784055828282',
'865784055828316','865784055828407','865784055891710','865784055891751',
'865784055891785','865784055891793','865784055891801','865784055891819',
'865784055891835','865784055891843','865784055893583','865784055893732',
'865784055893765','865784055893781','865784055893815','865784055893823',
'865784055893831','865784055893849','865784055893864','865784055894508',
'865784055899424','865784055899960','865784055900008','865784055900016',
'865784055900024','865784055900032','865784055900040','865784055900115',
'865784055900172','865784055900206','865784055900255','865784055900297',
'865784055900586','865784055901683','865784055901857','865784055902566',
'865784055902574','865784055904422','865784055904489','865784055906641',
'865784055906674','865784055906815','865784055906856','865784055906864',
'865784055910502','865784055912474','865784055912490','865784055912615',
'865784055912623','865784055912979','865784055913183','865784055913563',
'865784055913589','865784055913597','865784055913621','865784055913647',
'865784055913688','865784055913696','865784055913704','865784055913712',
'865784055913720','865784055913738','865784055913746','865784055913753',
'865784055913761','865784055913795','865784055920113','865784055920204',
'865784055940467','865784055943487','867019050264474','867019050284472']  # Add your actual IMEIs here

    # Define the date range
    start_date = "2025-01-20"
    end_date = "2025-01-20"

    # Fetch data from BigQuery for specified IMEIs within the date range
    data = read_data_from_bigquery(project_id, dataset_id, table_id, imei_list, start_date, end_date)

    # Define the output Excel file name
    excel_file_name = "vehicle_data_status.xlsx"

    # Save the results to an Excel file
    save_to_excel(data, excel_file_name)


Data saved to vehicle_data_status.xlsx


In [6]:
import pandas as pd

# Specify the file paths for IBM and GCP data
ibm_file_path = r'C:\Users\Ribish\Downloads\Karan\Karan\FUEL_MC2BMLRC0MF072783.xlsx'  # Replace with your actual IBM file path
gcp_file_path = r'C:\Users\Ribish\Downloads\VECV\fceetdata.xlsx'  # Replace with your actual GCP file path

# Load each file as a separate DataFrame
ibm_df = pd.read_excel(ibm_file_path)
gcp_df = pd.read_excel(gcp_file_path)

# Convert 'eDateTime' to datetime format in each DataFrame
ibm_df['eDateTime'] = pd.to_datetime(ibm_df['eDateTime'], dayfirst=True)
gcp_df['eDateTime'] = pd.to_datetime(gcp_df['eDateTime'], dayfirst=True)

# Define the date to analyze
date_to_analyze = '2024-10-28'

# Filter data for the specified date
ibm_filtered = ibm_df[ibm_df['eDateTime'].dt.date == pd.to_datetime(date_to_analyze).date()]
gcp_filtered = gcp_df[gcp_df['eDateTime'].dt.date == pd.to_datetime(date_to_analyze).date()]

# Find packets present in GCP but missing in IBM, and vice versa
missing_in_ibm = gcp_filtered[~gcp_filtered['eDateTime'].isin(ibm_filtered['eDateTime'])]
missing_in_gcp = ibm_filtered[~ibm_filtered['eDateTime'].isin(gcp_filtered['eDateTime'])]

print("Packets in GCP but missing in IBM:", missing_in_ibm)
print("Packets in IBM but missing in GCP:", missing_in_gcp)

# Merge on 'eDateTime' to identify matching packets
merged_df = pd.merge(ibm_filtered, gcp_filtered, on='eDateTime', suffixes=('_ibm', '_gcp'))

# Calculate time difference (delay) in seconds
merged_df['time_diff'] = (merged_df['eDateTime_gcp'] - merged_df['eDateTime_ibm']).dt.total_seconds()
average_delay = merged_df['time_diff'].mean()

print(f"Average delay between IBM and GCP: {average_delay} seconds")
print("Time differences (in seconds):", merged_df['time_diff'].describe())



Packets in GCP but missing in IBM: Empty DataFrame
Columns: [deviceId, eDateTime, eDate, packetName, deviceName, errorFlag, sdpArrivalTime, stage1ProcessingTime, stage2ProcessingTime, stage3ProcessingTime, dwIngestionTime, createdDate, updatedDate, schemaVersion, chassisSeries, chassisNumber, triggerType, latitude, longitude, heading, altitude, speed, totalDistance, totalEngineHours, totalFuelConsumption, brakeCount, coastingDistance, coastingTime, cruiseDistance, cruiseFuelConsumption, cruiseTime, fuelLevel, idleFuelConsumption, idleTime, movingFuelConsumption, movingTime, ptoFuelConsumption, ptoTime, stopCount, sweetSpotDistance, sweetSpotFuelConsumption, sweetSpotTime, topGearFuelConsumption, topGearTime, topGearDistance, totalAdBlueUsed]
Index: []

[0 rows x 46 columns]
Packets in IBM but missing in GCP: Empty DataFrame
Columns: [_id, _rev, createdDateTime, createdDate, receivedDateTime, receivedDate, vin, chassisSeries, chassisNumber, triggerType, longitude, latitude, heading, alt

KeyError: 'eDateTime_gcp'

In [37]:
import pandas as pd

# Specify the file paths for IBM and GCP data
ibm_file_path = r'C:\Users\Ribish\Downloads\Karan\Karan\FUEL_MC2BMLRC0MF072783.xlsx'
gcp_file_path = r'C:\Users\Ribish\Downloads\SortedFileName.xlsx'

# Load each file as a separate DataFrame
try:
    ibm_df = pd.read_excel(ibm_file_path)
    gcp_df = pd.read_excel(gcp_file_path)

    # Ensure eDateTime column is correctly formatted
    ibm_df['eDateTime'] = pd.to_datetime(ibm_df['eDateTime'], errors='coerce', dayfirst=True)
    gcp_df['eDateTime'] = pd.to_datetime(gcp_df['eDateTime'], errors='coerce', dayfirst=True)

    # Define the date to analyze
    date_to_analyze = '2024-10-28'

    # Filter data for the specific date
    ibm_filtered = ibm_df[ibm_df['eDateTime'].dt.date == pd.to_datetime(date_to_analyze).date()]
    gcp_filtered = gcp_df[gcp_df['eDateTime'].dt.date == pd.to_datetime(date_to_analyze).date()]

    # Identify packets present in GCP but missing in IBM, and vice versa
    missing_in_ibm = gcp_filtered[~gcp_filtered['eDateTime'].isin(ibm_filtered['eDateTime'])]
    missing_in_gcp = ibm_filtered[~ibm_filtered['eDateTime'].isin(gcp_filtered['eDateTime'])]

    print("Packets in GCP but missing in IBM:\n", missing_in_ibm)
    print("Packets in IBM but missing in GCP:\n", missing_in_gcp)

    # Merge on 'eDateTime' to identify matching packets
    merged_df = pd.merge(ibm_filtered, gcp_filtered, on='eDateTime', suffixes=('_ibm', '_gcp'))

    # Check if time difference calculation is needed
    if 'eDateTime_ibm' in merged_df.columns and 'eDateTime_gcp' in merged_df.columns:
        merged_df['time_diff'] = (merged_df['eDateTime_gcp'] - merged_df['eDateTime_ibm']).dt.total_seconds()
        average_delay = merged_df['time_diff'].mean()
        print(f"Average delay between IBM and GCP: {average_delay} seconds")
        print("Time differences (in seconds):\n", merged_df['time_diff'].describe())
    else:
        print("No matching packets to calculate time difference.")

except KeyError as e:
    print(f"Column error: {e}. Please verify that both files contain 'eDateTime' column.")
except Exception as ex:
    print(f"An error occurred: {ex}")


Packets in GCP but missing in IBM:
 Empty DataFrame
Columns: [deviceId, eDateTime, eDate, packetName, deviceName, errorFlag, sdpArrivalTime, stage1ProcessingTime, stage2ProcessingTime, stage3ProcessingTime, dwIngestionTime, createdDate, updatedDate, schemaVersion, chassisSeries, chassisNumber, triggerType, latitude, longitude, heading, altitude, speed, totalDistance, totalEngineHours, totalFuelConsumption, brakeCount, coastingDistance, coastingTime, cruiseDistance, cruiseFuelConsumption, cruiseTime, fuelLevel, idleFuelConsumption, idleTime, movingFuelConsumption, movingTime, ptoFuelConsumption, ptoTime, stopCount, sweetSpotDistance, sweetSpotFuelConsumption, sweetSpotTime, topGearFuelConsumption, topGearTime, topGearDistance, totalAdBlueUsed]
Index: []

[0 rows x 46 columns]
Packets in IBM but missing in GCP:
 Empty DataFrame
Columns: [_id, _rev, createdDateTime, createdDate, receivedDateTime, receivedDate, vin, chassisSeries, chassisNumber, triggerType, longitude, latitude, heading, a

In [38]:
import pandas as pd

# Specify the file paths for IBM and GCP data
ibm_file_path = r'C:\Users\Ribish\Downloads\Karan\Karan\FUEL_MC2BMLRC0MF072783.xlsx'  # IBM file path
gcp_file_path = r'C:\Users\Ribish\Downloads\SortedFileName.xlsx'  # GCP file path

# Load each file as a separate DataFrame
ibm_df = pd.read_excel(ibm_file_path)
gcp_df = pd.read_excel(gcp_file_path)

# Convert 'eDateTime' to datetime format in each DataFrame
ibm_df['eDateTime'] = pd.to_datetime(ibm_df['eDateTime'], dayfirst=True)
gcp_df['eDateTime'] = pd.to_datetime(gcp_df['eDateTime'], dayfirst=True)

# Define the date to analyze
date_to_analyze = '2024-10-28'

# Filter data for the specified date
ibm_filtered = ibm_df[ibm_df['eDateTime'].dt.date == pd.to_datetime(date_to_analyze).date()]
gcp_filtered = gcp_df[gcp_df['eDateTime'].dt.date == pd.to_datetime(date_to_analyze).date()]

# Find packets present in GCP but missing in IBM, and vice versa
missing_in_ibm = gcp_filtered[~gcp_filtered['eDateTime'].isin(ibm_filtered['eDateTime'])]
missing_in_gcp = ibm_filtered[~ibm_filtered['eDateTime'].isin(gcp_filtered['eDateTime'])]

print("Packets in GCP but missing in IBM:", missing_in_ibm)
print("Packets in IBM but missing in GCP:", missing_in_gcp)

# Merge on 'eDateTime' to identify matching packets
merged_df = pd.merge(ibm_filtered, gcp_filtered, on='eDateTime', suffixes=('_ibm', '_gcp'))

# Check if there are any matching packets before calculating time difference
if not merged_df.empty:
    # Calculate time difference (delay) in seconds
    merged_df['time_diff'] = (merged_df['eDateTime'] - merged_df['eDateTime']).dt.total_seconds()
    average_delay = merged_df['time_diff'].mean()

    print(f"Average delay between IBM and GCP: {average_delay} seconds")
    print("Time differences (in seconds):", merged_df['time_diff'].describe())
else:
    print("No matching packets to calculate time difference.")


Packets in GCP but missing in IBM: Empty DataFrame
Columns: [deviceId, eDateTime, eDate, packetName, deviceName, errorFlag, sdpArrivalTime, stage1ProcessingTime, stage2ProcessingTime, stage3ProcessingTime, dwIngestionTime, createdDate, updatedDate, schemaVersion, chassisSeries, chassisNumber, triggerType, latitude, longitude, heading, altitude, speed, totalDistance, totalEngineHours, totalFuelConsumption, brakeCount, coastingDistance, coastingTime, cruiseDistance, cruiseFuelConsumption, cruiseTime, fuelLevel, idleFuelConsumption, idleTime, movingFuelConsumption, movingTime, ptoFuelConsumption, ptoTime, stopCount, sweetSpotDistance, sweetSpotFuelConsumption, sweetSpotTime, topGearFuelConsumption, topGearTime, topGearDistance, totalAdBlueUsed]
Index: []

[0 rows x 46 columns]
Packets in IBM but missing in GCP: Empty DataFrame
Columns: [_id, _rev, createdDateTime, createdDate, receivedDateTime, receivedDate, vin, chassisSeries, chassisNumber, triggerType, longitude, latitude, heading, alt

In [34]:
import pandas as pd

# Specify the file paths for IBM and GCP data
ibm_file_path = r'C:\Users\Ribish\Downloads\Karan\Karan\FUEL_MC2BMLRC0MF072783.xlsx'  # IBM file path
gcp_file_path = r'C:\Users\Ribish\Downloads\VECV\fceetdata.xlsx'  # GCP file path

# Load each file as a separate DataFrame
ibm_df = pd.read_excel(ibm_file_path)
gcp_df = pd.read_excel(gcp_file_path)

# Convert 'eDateTime' to datetime format in each DataFrame
ibm_df['eDateTime'] = pd.to_datetime(ibm_df['eDateTime'], dayfirst=True)
gcp_df['eDateTime'] = pd.to_datetime(gcp_df['eDateTime'], dayfirst=True)

# Define the date to analyze
date_to_analyze = '2024-10-28'

# Filter data for the specified date
ibm_filtered = ibm_df[ibm_df['eDateTime'].dt.date == pd.to_datetime(date_to_analyze).date()]
gcp_filtered = gcp_df[gcp_df['eDateTime'].dt.date == pd.to_datetime(date_to_analyze).date()]

# Find packets present in GCP but missing in IBM, and vice versa
missing_in_ibm = gcp_filtered[~gcp_filtered['eDateTime'].isin(ibm_filtered['eDateTime'])]
missing_in_gcp = ibm_filtered[~ibm_filtered['eDateTime'].isin(gcp_filtered['eDateTime'])]

# Merge on 'eDateTime' to identify matching packets
merged_df = pd.merge(ibm_filtered, gcp_filtered, on='eDateTime', suffixes=('_ibm', '_gcp'))

# Check if there are any matching packets before calculating time difference
if not merged_df.empty:
    # Calculate time difference (delay) in seconds
    merged_df['time_diff'] = (merged_df['eDateTime'] - merged_df['eDateTime']).dt.total_seconds()
    average_delay = merged_df['time_diff'].mean()
    merged_df['average_delay'] = average_delay  # Add average delay as a new column
else:
    merged_df['time_diff'] = None
    print("No matching packets to calculate time difference.")

# Save the data to an Excel file with multiple sheets
output_file_path = r'C:\Users\Ribish\Downloads\comparison_results.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    missing_in_ibm.to_excel(writer, sheet_name='Missing in IBM', index=False)
    missing_in_gcp.to_excel(writer, sheet_name='Missing in GCP', index=False)
    merged_df.to_excel(writer, sheet_name='Matching Packets', index=False)

print(f"Comparison results saved to {output_file_path}")


Comparison results saved to C:\Users\Ribish\Downloads\comparison_results.xlsx


In [26]:
import pandas as pd

# Load the two Excel sheets
sheet1 = pd.read_excel(r'C:\Users\700023\Downloads\359207068358025bs6.xlsx')
sheet2 = pd.read_excel(r'C:\Users\700023\Downloads\359207068358025.xlsx')

# Merge both sheets based on the 'eDateTime' column
merged_sheet = pd.merge(sheet1, sheet2, on='eDateTime', how='inner')

# Save the merged result to a new Excel file
merged_sheet.to_excel('merged1_output.xlsx', index=False)

print("Sheets merged and saved to 'merged_output.xlsx'")


Sheets merged and saved to 'merged_output.xlsx'


In [17]:
import pandas as pd

# Define file paths
file1_path = r'C:\Users\700023\Downloads\Probs4.xlsx'
file2_path = r'C:\Users\700023\Downloads\Probs6.xlsx'

# Load all sheets from both Excel files
sheets1 = pd.read_excel(file1_path, sheet_name=None)
sheets2 = pd.read_excel(file2_path, sheet_name=None)

# Create an empty list to store merged DataFrames
merged_sheets = []

# Iterate through each sheet
for sheet_name in sheets1.keys():
    if sheet_name in sheets2:
        # Read corresponding sheets
        df1 = sheets1[sheet_name]
        df2 = sheets2[sheet_name]

        # Merge the sheets on 'deviceId' and 'eDateTime'
        merged_df = pd.merge(df1, df2, on=['deviceId', 'eDateTime'], how='inner')

        # Add merged DataFrame to the list
        merged_sheets.append((sheet_name, merged_df))

# Save merged sheets to a new Excel file
with pd.ExcelWriter('merged1_output.xlsx') as writer:
    for sheet_name, merged_df in merged_sheets:
        merged_df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Sheets merged and saved to 'merged1_output.xlsx'")


Sheets merged and saved to 'merged1_output.xlsx'


In [58]:
import pandas as pd

# Replace with the path to your Excel file
file_path = r'C:\Users\Ribish\Downloads\VECV\CVdata.xlsx'

# Load the Excel file
df = pd.read_excel(file_path)

# Convert 'eDateTime' column to datetime format if it's not already
df['eDateTime'] = pd.to_datetime(df['eDateTime'], errors='coerce')

# Sort the DataFrame by 'eDateTime' in ascending order
df_sorted = df.sort_values(by='eDateTime').reset_index(drop=True)

# Save the sorted DataFrame back to Excel
sorted_file_path = r'C:\Users\Ribish\Downloads\23l.xlsx'
df_sorted.to_excel(sorted_file_path, index=False)

print(f"Data sorted by 'eDateTime' and saved to {sorted_file_path}")


Data sorted by 'eDateTime' and saved to C:\Users\Ribish\Downloads\23l.xlsx


In [62]:




import pandas as pd

# Load the two Excel files
file1_path = (r'C:\Users\Ribish\Downloads\Karan\Karan\FUEL_MC2M6GRC0LC064500.xlsx')
file2_path = (r'C:\Users\Ribish\Downloads\23R.xlsx')


# Read both files, setting `AW` as `eDate` in the first file and `B` as `eDate` in the second
df1 = pd.read_excel(file1_path, usecols=['AW', '...other_columns'], names=['eDate', '...other_columns'])
df2 = pd.read_excel(file2_path, usecols=['B', '...other_columns'], names=['eDate', '...other_columns'])

# Continue with merging and processing, assuming both now have 'eDate' as the column name
# Perform the merge on 'eDate' (or 'eDateTime' if you have timestamp details in this column)
merged_df = pd.merge(df1, df2, on='eDate', suffixes=('_file1', '_file2'))

# Calculate time difference or any other analysis you need
if 'eDateTime_file1' in merged_df.columns and 'eDateTime_file2' in merged_df.columns:
    merged_df['time_diff'] = (merged_df['eDateTime_file1'] - merged_df['eDateTime_file2']).dt.total_seconds()
    average_delay = merged_df['time_diff'].mean()
    print("Average delay:", average_delay)
else:
    print("eDateTime columns missing in one of the files.")

# Save the merged DataFrame if needed
output_path = 'C:/Users/Ribish/Downloads/merged_comparison.xlsx'
merged_df.to_excel(output_path, index=False)
print(f"Merged data saved to {output_path}")



ValueError: Usecols do not match columns, columns expected but not found: ['AW'] (sheet: 0)

In [ ]:
import pandas as pd

# Specify the file paths for IBM and GCP data
ibm_file_path = r'C:\Users\Ribish\Downloads\Karan\Karan\FUEL_MC2BMLRC0MF072783.xlsx'  # Replace with your actual IBM file path
gcp_file_path = r'C:\Users\Ribish\Downloads\VECV\fceetdata.xlsx'  # Replace with your actual GCP file path

# Load each file as a separate DataFrame
ibm_df = pd.read_excel(ibm_file_path)
gcp_df = pd.read_excel(gcp_file_path)

# Convert 'eDateTime' to datetime format in each DataFrame
ibm_df['eDateTime'] = pd.to_datetime(ibm_df['eDateTime'], dayfirst=True)
gcp_df['eDateTime'] = pd.to_datetime(gcp_df['eDateTime'], dayfirst=True)

# Define the date to analyze
date_to_analyze = '2024-10-28'

# Filter data for the specified date
ibm_filtered = ibm_df[ibm_df['eDateTime'].dt.date == pd.to_datetime(date_to_analyze).date()]
gcp_filtered = gcp_df[gcp_df['eDateTime'].dt.date == pd.to_datetime(date_to_analyze).date()]

# Find packets present in GCP but missing in IBM, and vice versa
missing_in_ibm = gcp_filtered[~gcp_filtered['eDateTime'].isin(ibm_filtered['eDateTime'])]
missing_in_gcp = ibm_filtered[~ibm_filtered['eDateTime'].isin(gcp_filtered['eDateTime'])]

print("Packets in GCP but missing in IBM:", missing_in_ibm)
print("Packets in IBM but missing in GCP:", missing_in_gcp)

# Merge on 'eDateTime' to identify matching packets
merged_df = pd.merge(ibm_filtered, gcp_filtered, on='eDateTime', suffixes=('_ibm', '_gcp'))

# Calculate time difference (delay) in seconds
merged_df['time_diff'] = (merged_df['eDateTime_gcp'] - merged_df['eDateTime_ibm']).dt.total_seconds()
average_delay = merged_df['time_diff'].mean()

print(f"Average delay between IBM and GCP: {average_delay} seconds")
print("Time differences (in seconds):", merged_df['time_diff'].describe())



Packets in GCP but missing in IBM: Empty DataFrame
Columns: [deviceId, eDateTime, eDate, packetName, deviceName, errorFlag, sdpArrivalTime, stage1ProcessingTime, stage2ProcessingTime, stage3ProcessingTime, dwIngestionTime, createdDate, updatedDate, schemaVersion, chassisSeries, chassisNumber, triggerType, latitude, longitude, heading, altitude, speed, totalDistance, totalEngineHours, totalFuelConsumption, brakeCount, coastingDistance, coastingTime, cruiseDistance, cruiseFuelConsumption, cruiseTime, fuelLevel, idleFuelConsumption, idleTime, movingFuelConsumption, movingTime, ptoFuelConsumption, ptoTime, stopCount, sweetSpotDistance, sweetSpotFuelConsumption, sweetSpotTime, topGearFuelConsumption, topGearTime, topGearDistance, totalAdBlueUsed]
Index: []

[0 rows x 46 columns]
Packets in IBM but missing in GCP: Empty DataFrame
Columns: [_id, _rev, createdDateTime, createdDate, receivedDateTime, receivedDate, vin, chassisSeries, chassisNumber, triggerType, longitude, latitude, heading, alt

KeyError: 'eDateTime_gcp'

In [39]:
import pandas as pd

# Load the Excel sheets
excel1 = pd.read_excel(r'C:\Users\Ribish\Downloads\Karan\Karan\FUEL_MC2BMLRC0MF072783.xlsx')
excel2 = pd.read_excel( r'C:\Users\Ribish\Downloads\VECV\fceetdata.xlsx')

# Standardize `eDateTime` format
excel1['eDateTime'] = pd.to_datetime(excel1['eDateTime'])
excel2['eDateTime'] = pd.to_datetime(excel2['eDateTime'])

# Check for missing entries by comparing `eDateTime`
missing_in_excel2 = excel1[~excel1['eDateTime'].isin(excel2['eDateTime'])]
missing_in_excel1 = excel2[~excel2['eDateTime'].isin(excel1['eDateTime'])]

# Generate summary statistics
summary = {
    "Total in Excel1": [len(excel1)],
    "Total in Excel2": [len(excel2)],
    "Missing in Excel2": [len(missing_in_excel2)],
    "Missing in Excel1": [len(missing_in_excel1)],
    "Duplicate entries in Excel1": [excel1['eDateTime'].duplicated().sum()],
    "Duplicate entries in Excel2": [excel2['eDateTime'].duplicated().sum()],
}

# Convert summary dictionary to DataFrame
summary_df = pd.DataFrame(summary)

# Save the results to a new Excel file with multiple sheets
with pd.ExcelWriter("analysis_results.xlsx") as writer:
    summary_df.to_excel(writer, sheet_name="Summary", index=False)
    missing_in_excel2.to_excel(writer, sheet_name="Missing in Excel2", index=False)
    missing_in_excel1.to_excel(writer, sheet_name="Missing in Excel1", index=False)

print("Analysis complete. Results saved to 'analysis_results.xlsx'")


Analysis complete. Results saved to 'analysis_results.xlsx'


In [41]:
import pandas as pd

# Load the Excel sheets
excel1 = pd.read_excel(r'C:\Users\Ribish\Downloads\Karan\Karan\FUEL_MC2BMLRC0MF072783.xlsx')
excel2 = pd.read_excel( r'C:\Users\Ribish\Downloads\VECV\fceetdata.xlsx')

# Standardize `eDateTime` format
excel1['eDateTime'] = pd.to_datetime(excel1['eDateTime'])
excel2['eDateTime'] = pd.to_datetime(excel2['eDateTime'])

# Check for missing entries by comparing `eDateTime`
missing_in_excel2 = excel1[~excel1['eDateTime'].isin(excel2['eDateTime'])]
missing_in_excel1 = excel2[~excel2['eDateTime'].isin(excel1['eDateTime'])]

# Find duplicate entries in each sheet based on `eDateTime`
duplicates_in_excel1 = excel1[excel1.duplicated(subset=['eDateTime'], keep=False)]
duplicates_in_excel2 = excel2[excel2.duplicated(subset=['eDateTime'], keep=False)]

# Generate summary statistics
summary = {
    "Total in Excel1": [len(excel1)],
    "Total in Excel2": [len(excel2)],
    "Missing in Excel2": [len(missing_in_excel2)],
    "Missing in Excel1": [len(missing_in_excel1)],
    "Duplicate entries in Excel1": [len(duplicates_in_excel1)],
    "Duplicate entries in Excel2": [len(duplicates_in_excel2)],
}

# Convert summary dictionary to DataFrame
summary_df = pd.DataFrame(summary)

# Save the results to a new Excel file with multiple sheets
with pd.ExcelWriter("analysis_results_with_duplicates.xlsx") as writer:
    summary_df.to_excel(writer, sheet_name="Summary", index=False)
    missing_in_excel2.to_excel(writer, sheet_name="Missing in Excel2", index=False)
    missing_in_excel1.to_excel(writer, sheet_name="Missing in Excel1", index=False)
    duplicates_in_excel1.to_excel(writer, sheet_name="Duplicates in Excel1", index=False)
    duplicates_in_excel2.to_excel(writer, sheet_name="Duplicates in Excel2", index=False)

print("Analysis complete. Results saved to 'analysis_results_with_duplicates.xlsx'")


Analysis complete. Results saved to 'analysis_results_with_duplicates.xlsx'


In [44]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Load the Excel sheet
file_path = r"C:\Users\Ribish\Downloads\SFileName.xlsx"

# Identify duplicates in the `eDateTime` column
df['is_duplicate'] = df.duplicated(subset=['eDateTime'], keep=False)

# Save the modified DataFrame back to the Excel file (temporarily to work with openpyxl)
df.to_excel(file_path, index=False)

# Load the workbook with openpyxl to apply formatting
wb = load_workbook(file_path)
ws = wb.active

# Define colors for highlighting
green_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Green for unique
red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")    # Red for duplicates

# Get the column index for `eDateTime`
eDateTime_col = df.columns.get_loc("eDateTime") + 1  # +1 because openpyxl is 1-indexed

# Apply color based on duplicate status
for row in range(2, len(df) + 2):  # Start from row 2 to skip the header
    cell = ws.cell(row=row, column=eDateTime_col)
    if df.at[row - 2, 'is_duplicate']:  # Row - 2 to match DataFrame indexing
        cell.fill = red_fill
    else:
        cell.fill = green_fill

# Save the updated workbook
wb.save("highlighted_duplicates.xlsx")
wb.close()

print("Duplicates highlighted and saved to 'highlighted_duplicates.xlsx'")


Duplicates highlighted and saved to 'highlighted_duplicates.xlsx'


In [45]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Load the Excel file into a DataFrame
file_path = r"C:\Users\Ribish\Downloads\SFileName.xlsx"
df = pd.read_excel(file_path)

# Identify the first occurrence and repetitions in the `eDateTime` column
df['is_first_occurrence'] = ~df.duplicated(subset=['eDateTime'], keep='first')
df['is_duplicate'] = df.duplicated(subset=['eDateTime'], keep=False)

# Save the modified DataFrame temporarily back to the Excel file
df.to_excel(file_path, index=False)

# Load the workbook with openpyxl to apply formatting
wb = load_workbook(file_path)
ws = wb.active

# Define colors for highlighting
green_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Green for first occurrences
red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")    # Red for subsequent duplicates

# Get the column index for `eDateTime`
eDateTime_col = df.columns.get_loc("eDateTime") + 1  # +1 because openpyxl is 1-indexed

# Apply color based on first occurrence and duplicates
for row in range(2, len(df) + 2):  # Start from row 2 to skip the header
    cell = ws.cell(row=row, column=eDateTime_col)
    if df.at[row - 2, 'is_first_occurrence']:
        cell.fill = green_fill
    elif df.at[row - 2, 'is_duplicate']:
        cell.fill = red_fill

# Save the updated workbook
wb.save("highlighted_duplicates_first_occurrence.xlsx")
wb.close()

print("Duplicates highlighted and saved to 'highlighted_duplicates_first_occurrence.xlsx'")


Duplicates highlighted and saved to 'highlighted_duplicates_first_occurrence.xlsx'


In [46]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.chart import PieChart, Reference

# Load the Excel file into a DataFrame
file_path = r"C:\Users\Ribish\Downloads\SFileName.xlsx"
df = pd.read_excel(file_path)

# Identify first occurrence and repetitions in the `eDateTime` column
df['is_first_occurrence'] = ~df.duplicated(subset=['eDateTime'], keep='first')
df['is_duplicate'] = df.duplicated(subset=['eDateTime'], keep=False)

# Count green and red markings
green_count = df['is_first_occurrence'].sum()
red_count = df['is_duplicate'].sum() - green_count  # Total duplicates minus first occurrences

# Save the modified DataFrame temporarily back to the Excel file
df.to_excel(file_path, index=False)

# Load the workbook with openpyxl to apply formatting
wb = load_workbook(file_path)
ws = wb.active

# Define colors for highlighting
green_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Green for first occurrences
red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")    # Red for subsequent duplicates

# Get the column index for `eDateTime`
eDateTime_col = df.columns.get_loc("eDateTime") + 1  # +1 because openpyxl is 1-indexed

# Apply color based on first occurrence and duplicates
for row in range(2, len(df) + 2):  # Start from row 2 to skip the header
    cell = ws.cell(row=row, column=eDateTime_col)
    if df.at[row - 2, 'is_first_occurrence']:
        cell.fill = green_fill
    elif df.at[row - 2, 'is_duplicate']:
        cell.fill = red_fill

# Add a summary table for green and red counts
summary_row = len(df) + 4  # Place it a few rows after the data
ws.cell(row=summary_row, column=1, value="Color")
ws.cell(row=summary_row, column=2, value="Count")
ws.cell(row=summary_row + 1, column=1, value="Green (First Occurrences)")
ws.cell(row=summary_row + 1, column=2, value=green_count)
ws.cell(row=summary_row + 2, column=1, value="Red (Subsequent Duplicates)")
ws.cell(row=summary_row + 2, column=2, value=red_count)

# Create a pie chart for the green and red counts
pie = PieChart()
labels = Reference(ws, min_col=1, min_row=summary_row + 1, max_row=summary_row + 2)
data = Reference(ws, min_col=2, min_row=summary_row, max_row=summary_row + 2)
pie.add_data(data, titles_from_data=True)
pie.set_categories(labels)
pie.title = "eDateTime Duplicate Analysis"

# Place the pie chart in the Excel sheet
ws.add_chart(pie, f"D{summary_row}")

# Save the updated workbook
wb.save("highlighted_duplicates_with_summary.xlsx")
wb.close()

print("Duplicates highlighted and summary saved to 'highlighted_duplicates_with_summary.xlsx'")


Duplicates highlighted and summary saved to 'highlighted_duplicates_with_summary.xlsx'


In [47]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.chart import PieChart, Reference

# Load the Excel file into a DataFrame
file_path = r"C:\Users\Ribish\Downloads\SFileName.xlsx"
df = pd.read_excel(file_path)

# Mark first occurrence and subsequent duplicates
df['is_first_occurrence'] = ~df.duplicated(subset=['eDateTime'], keep='first')
df['is_duplicate'] = df.duplicated(subset=['eDateTime'], keep=False)

# Count green and red markings
green_count = df['is_first_occurrence'].sum()  # First occurrences including unique entries
red_count = df['is_duplicate'].sum() - green_count  # Total duplicates minus first occurrences

# Verify total
if green_count + red_count == len(df):
    print("Counts verified correctly.")
else:
    print("Mismatch in counts.")

# Load the workbook with openpyxl to apply formatting
wb = load_workbook(file_path)
ws = wb.active

# Define colors for highlighting
green_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Green for first occurrences
red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")    # Red for subsequent duplicates

# Get the column index for `eDateTime`
eDateTime_col = df.columns.get_loc("eDateTime") + 1  # +1 because openpyxl is 1-indexed

# Apply color based on first occurrence and duplicates
for row in range(2, len(df) + 2):  # Start from row 2 to skip the header
    cell = ws.cell(row=row, column=eDateTime_col)
    if df.at[row - 2, 'is_first_occurrence']:
        cell.fill = green_fill
    elif df.at[row - 2, 'is_duplicate']:
        cell.fill = red_fill

# Add a summary table for green and red counts
summary_row = len(df) + 4  # Place it a few rows after the data
ws.cell(row=summary_row, column=1, value="Color")
ws.cell(row=summary_row, column=2, value="Count")
ws.cell(row=summary_row + 1, column=1, value="Green (First Occurrences)")
ws.cell(row=summary_row + 1, column=2, value=green_count)
ws.cell(row=summary_row + 2, column=1, value="Red (Subsequent Duplicates)")
ws.cell(row=summary_row + 2, column=2, value=red_count)

# Create a pie chart for the green and red counts
pie = PieChart()
labels = Reference(ws, min_col=1, min_row=summary_row + 1, max_row=summary_row + 2)
data = Reference(ws, min_col=2, min_row=summary_row, max_row=summary_row + 2)
pie.add_data(data, titles_from_data=True)
pie.set_categories(labels)
pie.title = "eDateTime Duplicate Analysis"

# Place the pie chart in the Excel sheet
ws.add_chart(pie, f"D{summary_row}")

# Save the updated workbook
wb.save("highlighted_duplicates_with_summary_corrected.xlsx")
wb.close()

print("Duplicates highlighted and summary saved to 'highlighted_duplicates_with_summary_corrected.xlsx'")


Mismatch in counts.
Duplicates highlighted and summary saved to 'highlighted_duplicates_with_summary_corrected.xlsx'


In [ ]:
from google.cloud import bigquery

from google.api_core.exceptions import GoogleAPIError

import pandas as pd



def read_data_from_bigquery(project_id, dataset_id, table_id, imei_list, start_date, end_date):

    # Construct a BigQuery client object

    client = bigquery.Client()



    # Create a query string to fetch data for the given IMEIs within the specified date range

    imei_str = ', '.join(f"'{imei}'" for imei in imei_list)  # Formatting IMEIs for SQL query

    query = f"""

        SELECT * 

        FROM `{project_id}.{dataset_id}.{table_id}`

        WHERE deviceId IN ({imei_str})

        AND DATE(eDate) BETWEEN '{start_date}' AND '{end_date}';

    """



    try:

        # Run the query and get the result

        query_job = client.query(query)

        results = query_job.result()



        # Collect and return the rows as a list of dictionaries

        rows = [dict(row) for row in results]

        return rows



    except GoogleAPIError as e:

        print("An error occurred:", e)

        return []



def save_to_excel(data, file_name):

    # Convert data to a DataFrame

    df = pd.DataFrame(data)

    print(f"Number of columns: {len(df.columns)}")

    # Save the DataFrame to an Excel file

    df.to_excel(file_name, index=False)

    print(f"Column names: {', '.join(df.columns)}")

    print(f"Data saved to {file_name}")



if __name__ == "__main__":

    # Replace with your project_id, dataset_id, and table_id

    project_id = "conn-datalake-prod"

    dataset_id = "bronze_telematics"

    table_id = "canbs4_pkt"  # Update with your actual table ID



    # List of IMEIs to fetch data for

    imei_list = ['352914090435716']

    

    # Define the date range

    start_date = '2024-11-18'

    end_date = '2024-11-20'



    # Fetch data from BigQuery for specified IMEIs within the date range

    data = read_data_from_bigquery(project_id, dataset_id, table_id, imei_list, start_date, end_date)



    # Check if data was fetched successfully

    if data:

        # Define the output Excel file name

        excel_file_name = "bgqcan.xlsx"



        # Save the data to an Excel file

        save_to_excel(data, excel_file_name)



    else:

        print("No data found.")
from google.cloud import bigquery

from google.api_core.exceptions import GoogleAPIError

import pandas as pd



def read_data_from_bigquery(project_id, dataset_id, table_id, imei_list, start_date, end_date):

    # Construct a BigQuery client object

    client = bigquery.Client()



    # Create a query string to fetch data for the given IMEIs within the specified date range

    imei_str = ', '.join(f"'{imei}'" for imei in imei_list)  # Formatting IMEIs for SQL query

    query = f"""

        SELECT * 

        FROM `{project_id}.{dataset_id}.{table_id}`

        WHERE deviceId IN ({imei_str})

        AND DATE(eDate) BETWEEN '{start_date}' AND '{end_date}';

    """



    try:

        # Run the query and get the result

        query_job = client.query(query)

        results = query_job.result()



        # Collect and return the rows as a list of dictionaries

        rows = [dict(row) for row in results]

        return rows



    except GoogleAPIError as e:

        print("An error occurred:", e)

        return []



def save_to_excel(data, file_name):

    # Convert data to a DataFrame

    df = pd.DataFrame(data)

    print(f"Number of columns: {len(df.columns)}")

    # Save the DataFrame to an Excel file

    df.to_excel(file_name, index=False)

    print(f"Column names: {', '.join(df.columns)}")

    print(f"Data saved to {file_name}")



if __name__ == "__main__":

    # Replace with your project_id, dataset_id, and table_id

    project_id = "conn-datalake-prod"

    dataset_id = "bronze_telematics"

    table_id = "canbs4_pkt"  # Update with your actual table ID



    # List of IMEIs to fetch data for

    imei_list = ['352914090435716']

    

    # Define the date range

    start_date = '2024-11-18'

    end_date = '2024-11-20'



    # Fetch data from BigQuery for specified IMEIs within the date range

    data = read_data_from_bigquery(project_id, dataset_id, table_id, imei_list, start_date, end_date)



    # Check if data was fetched successfully

    if data:

        # Define the output Excel file name

        excel_file_name = "bgqcan.xlsx"



        # Save the data to an Excel file

        save_to_excel(data, excel_file_name)



    else:

        print("No data found.")
